# Section 4: Experiment Tracking（実験管理）

## 🎯 このセクションの目標
- **Snowflake ML Experiments** を使った実験管理を学ぶ
- 複数モデルの比較実験を行う
- パラメータ・メトリクスを記録して追跡する

## 📋 Experiment Trackingとは？
機械学習の実験を体系的に管理するための機能です：
- モデルのパラメータと結果を記録
- 複数の実験を比較
- 再現性のある実験環境を構築

---

## 💡 ハンズオンでの位置づけ

このハンズオンでは、**各機能を独立して学ぶ**ことを目的としています。

| Section | 学習目的 |
|---------|----------|
| 03 Model Training | ハイパラ**自動探索**（RandomizedSearchCV） |
| 04 Experiment Tracking | 実験の**記録・比較**機能 |

そのため、03で見つけた最良パラメータとは**別に**、異なるパラメータセットで実験を行い、Experiment Trackingの機能を体験します。

> 📝 **実運用では**: RandomizedSearchCVの各イテレーション結果をExperiment Trackingに記録し、一元管理することが多いです。


---
## 4.1 環境設定

> 💡 **パッケージについて**: `environment.yml` により必要なパッケージ（snowflake-ml-python等）は自動的にインストールされています。


In [ ]:
# Warningを抑制（ハンズオン向け）
import warnings
warnings.filterwarnings('ignore')

from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *

# ML関連（sklearn版を使用 - コンテナランタイム互換性のため）
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import pandas as pd
import numpy as np
import json

# 可視化
import matplotlib.pyplot as plt

# SHAP（モデル解釈性）
import shap

# Experiment Tracking
from snowflake.ml.experiment import ExperimentTracking

# セッションの取得
session = get_active_session()

# 環境設定
session.use_database("MLOPS_HOL_DB")
session.use_schema("FEATURE_STORE")
session.use_warehouse("MLOPS_HOL_SQL_WH")

print("✅ ライブラリのインポート完了")


---
## 4.2 Experiment Trackingの初期化


In [ ]:
# ExperimentTracking のインスタンス化
exp = ExperimentTracking(session=session)

# 実験名を設定
EXPERIMENT_NAME = "CHURN_PREDICTION_EXPERIMENT"
exp.set_experiment(EXPERIMENT_NAME)

print(f"✅ Experiment設定完了: {EXPERIMENT_NAME}")


---
## 4.3 学習データの準備


In [ ]:
# 学習データセットを読み込み
training_data = session.table("TRAINING_DATASET_V1")

# 特徴量とラベルの定義
FEATURE_COLS = [
    "DAYS_SINCE_LAST_ORDER",
    "TOTAL_ORDER_COUNT",
    "ORDER_COUNT_2024_H1",
    "TOTAL_ORDER_AMOUNT",
    "AVG_ORDER_AMOUNT",
    "TOTAL_AMOUNT_2024_H1",
    "RETURN_RATE"
]
LABEL_COL = "IS_CHURNED"
ID_COL = "CUSTOMER_ID"

# データの分割
model_data = training_data.select([ID_COL] + FEATURE_COLS + [LABEL_COL])
train_sf, test_sf = model_data.random_split(weights=[0.8, 0.2], seed=42)

# Pandas DataFrameに変換（sklearn用）
train_pd = train_sf.to_pandas()
test_pd = test_sf.to_pandas()

X_train = train_pd[FEATURE_COLS]
y_train = train_pd[LABEL_COL]
X_test = test_pd[FEATURE_COLS]
y_test = test_pd[LABEL_COL]

print(f"訓練データ: {len(train_pd):,} 件")
print(f"テストデータ: {len(test_pd):,} 件")


---
## 4.4 実験の実行

複数のハイパーパラメータセットでモデルを訓練し、結果を記録します。

| Run | 特徴 | 期待 |
|-----|------|------|
| Baseline | デフォルトパラメータ | 基準モデル |
| DeepTree | 深い木 (max_depth=10) | 複雑なパターン学習 |
| Conservative | 浅い木 + 低学習率 | 過学習防止 |

### 📝 記録する情報

| 種類 | 内容 | 記録方法 |
|------|------|----------|
| パラメータ | ハイパーパラメータ | `log_param()` |
| メトリクス | Accuracy, F1等 | `log_metric()` |
| 特徴量重要度 | XGBoost Feature Importance | `log_metric()` |
| SHAP値 | 平均絶対SHAP値 | `log_metric()` |
| 画像 | SHAP Summary Plot | `log_artifact()` |


In [ ]:
# 実験設定の定義
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

experiments = [
    {
        "run_name": f"Baseline_{timestamp}",
        "display_name": "Baseline",
        "params": {
            "max_depth": 5,
            "learning_rate": 0.1,
            "n_estimators": 100,
            "scale_pos_weight": 2
        }
    },
    {
        "run_name": f"DeepTree_{timestamp}",
        "display_name": "DeepTree",
        "params": {
            "max_depth": 10,
            "learning_rate": 0.1,
            "n_estimators": 100,
            "scale_pos_weight": 2
        }
    },
    {
        "run_name": f"Conservative_{timestamp}",
        "display_name": "Conservative",
        "params": {
            "max_depth": 3,
            "learning_rate": 0.05,
            "n_estimators": 200,
            "scale_pos_weight": 3
        }
    }
]

print(f"✅ {len(experiments)}つの実験を定義しました")
print(f"   タイムスタンプ: {timestamp}")

In [ ]:
# 各実験の実行とログ記録
results = []

for experiment in experiments:
    run_name = experiment["run_name"]
    display_name = experiment["display_name"]
    params = experiment["params"]
    
    print(f"\n{'='*60}")
    print(f"Running: {display_name}")
    print(f"{'='*60}")
    
    # Experiment Trackingでランを開始
    with exp.start_run(run_name=run_name):
        
        # パラメータをログ
        exp.log_param("model_type", "XGBoost")
        exp.log_param("display_name", display_name)
        for param_name, param_value in params.items():
            exp.log_param(param_name, param_value)
        
        # モデルの訓練（sklearn版XGBoost）
        model = xgb.XGBClassifier(
            random_state=42,
            use_label_encoder=False,
            eval_metric='logloss',
            **params
        )
        model.fit(X_train, y_train)
        
        # 予測と評価
        y_pred = model.predict(X_test)
        
        # メトリクス計算
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # メトリクスをログ
        exp.log_metric("accuracy", accuracy)
        exp.log_metric("precision", precision)
        exp.log_metric("recall", recall)
        exp.log_metric("f1_score", f1)
        
        # =====================================================
        # 🆕 特徴量重要度（Feature Importance）を記録
        # =====================================================
        feature_importance = model.feature_importances_
        importance_dict = dict(zip(FEATURE_COLS, [float(v) for v in feature_importance]))
        
        # 各特徴量の重要度を個別のメトリクスとして記録
        for feature_name, importance in importance_dict.items():
            exp.log_metric(f"importance_{feature_name}", importance)
        
        print(f"\n  📊 特徴量重要度 (XGBoost):")
        for feat, imp in sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)[:3]:
            print(f"    {feat}: {imp:.4f}")
        
        # =====================================================
        # 🆕 SHAP値の計算と記録
        # =====================================================
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_test)
        
        # SHAP値の平均絶対値（グローバル重要度）を計算・記録
        shap_importance = np.abs(shap_values).mean(axis=0)
        shap_dict = dict(zip(FEATURE_COLS, [float(v) for v in shap_importance]))
        
        for feature_name, shap_imp in shap_dict.items():
            exp.log_metric(f"shap_{feature_name}", shap_imp)
        
        print(f"\n  🔍 SHAP値 (平均絶対値):")
        for feat, shap_val in sorted(shap_dict.items(), key=lambda x: x[1], reverse=True)[:3]:
            print(f"    {feat}: {shap_val:.4f}")
        
        # =====================================================
        # 🆕 SHAP Summary Plotを画像として保存・アーティファクト記録
        # =====================================================
        plt.figure(figsize=(10, 6))
        shap.summary_plot(shap_values, X_test, feature_names=FEATURE_COLS, show=False)
        plt.title(f"SHAP Summary Plot - {display_name}")
        plt.tight_layout()
        
        # 画像を保存
        shap_plot_path = f"/tmp/shap_summary_{display_name}.png"
        plt.savefig(shap_plot_path, bbox_inches='tight', dpi=150)
        plt.close()
        
        # アーティファクトとして記録
        exp.log_artifact(shap_plot_path)
        print(f"\n  📸 SHAP Plotを保存: {shap_plot_path}")
        
        # 結果を保存
        results.append({
            "run_name": display_name,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "model": model,
            "params": params,
            "feature_importance": importance_dict,
            "shap_importance": shap_dict,
            "shap_values": shap_values
        })
        
        print(f"\n  ✅ メトリクス:")
        print(f"    Accuracy:  {accuracy:.4f}")
        print(f"    Precision: {precision:.4f}")
        print(f"    Recall:    {recall:.4f}")
        print(f"    F1 Score:  {f1:.4f}")

print(f"\n✅ {len(experiments)}つの実験が完了しました")

---
## 4.5 実験結果の比較


In [ ]:
# 結果をDataFrameで比較
results_df = pd.DataFrame([{
    "Run": r["run_name"],
    "Accuracy": f"{r['accuracy']:.4f}",
    "Precision": f"{r['precision']:.4f}",
    "Recall": f"{r['recall']:.4f}",
    "F1 Score": f"{r['f1_score']:.4f}"
} for r in results])

print("=" * 70)
print("📊 実験結果の比較")
print("=" * 70)
print(results_df.to_string(index=False))
print("=" * 70)


In [ ]:
# 最良モデルの選択（F1スコア基準）
best_result = max(results, key=lambda x: x["f1_score"])

print("=" * 60)
print("🏆 最良モデル（F1スコア基準）")
print("=" * 60)
print(f"  Run Name:  {best_result['run_name']}")
print(f"  F1 Score:  {best_result['f1_score']:.4f}")
print(f"  Accuracy:  {best_result['accuracy']:.4f}")
print(f"  Precision: {best_result['precision']:.4f}")
print(f"  Recall:    {best_result['recall']:.4f}")
print("="  * 60)
print(f"  パラメータ: {best_result['params']}")
print("=" * 60)

# 最良モデルの特徴量重要度を表示
print("\n📊 最良モデルの特徴量重要度（上位5件）:")
sorted_importance = sorted(best_result['feature_importance'].items(), key=lambda x: x[1], reverse=True)
for feat, imp in sorted_importance[:5]:
    print(f"  {feat}: {imp:.4f}")

print("\n🔍 最良モデルのSHAP値（上位5件）:")
sorted_shap = sorted(best_result['shap_importance'].items(), key=lambda x: x[1], reverse=True)
for feat, shap_val in sorted_shap[:5]:
    print(f"  {feat}: {shap_val:.4f}")

# 最良モデルを保存
best_model = best_result["model"]
best_metrics = {
    "accuracy": best_result["accuracy"],
    "precision": best_result["precision"],
    "recall": best_result["recall"],
    "f1_score": best_result["f1_score"]
}


---
## 4.6 可視化


In [ ]:
import matplotlib.pyplot as plt

# 実験結果のバーチャート
metrics = ["accuracy", "precision", "recall", "f1_score"]
run_names = [r["run_name"] for r in results]

fig, axes = plt.subplots(1, 4, figsize=(14, 4))

for i, metric in enumerate(metrics):
    values = [r[metric] for r in results]
    colors = ['gold' if r["run_name"] == best_result["run_name"] else 'steelblue' for r in results]
    axes[i].bar(run_names, values, color=colors)
    axes[i].set_title(metric.replace('_', ' ').title())
    axes[i].set_ylim(0, 1)
    axes[i].tick_params(axis='x', rotation=45)

plt.suptitle('Experiment Results Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# 最良モデルのSHAP Summary Plotを表示
print(f"=== 🔍 最良モデル（{best_result['run_name']}）のSHAP Summary Plot ===")

plt.figure(figsize=(10, 6))
shap.summary_plot(
    best_result['shap_values'], 
    X_test, 
    feature_names=FEATURE_COLS,
    show=False
)
plt.title(f"SHAP Summary Plot - {best_result['run_name']}", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 このプロットの見方:")
print("   - 横軸: SHAP値（予測への影響度）")
print("   - 縦軸: 特徴量（重要度順）")
print("   - 色: 特徴量の値（赤=高、青=低）")


---
## 4.7 実験結果の永続化


In [ ]:
# 実験結果をテーブルに保存（補助的な記録）
import datetime

experiment_records = []
for r in results:
    experiment_records.append({
        "EXPERIMENT_NAME": EXPERIMENT_NAME,
        "RUN_NAME": r["run_name"],
        "ACCURACY": r["accuracy"],
        "PRECISION": r["precision"],
        "RECALL": r["recall"],
        "F1_SCORE": r["f1_score"],
        "PARAMS": json.dumps(r["params"]),
        "FEATURE_IMPORTANCE": json.dumps(r["feature_importance"]),  # 🆕 特徴量重要度
        "SHAP_IMPORTANCE": json.dumps(r["shap_importance"]),        # 🆕 SHAP値
        "CREATED_AT": datetime.datetime.now().isoformat()
    })

records_df = session.create_dataframe(experiment_records)
records_df.write.save_as_table(
    "MLOPS_HOL_DB.FEATURE_STORE.EXPERIMENT_RESULTS",
    mode="overwrite"
)

print("✅ 実験結果をテーブルに保存しました")
print("   テーブル: MLOPS_HOL_DB.FEATURE_STORE.EXPERIMENT_RESULTS")
print("\n📝 保存された項目:")
print("   - メトリクス（Accuracy, Precision, Recall, F1）")
print("   - パラメータ（JSON）")
print("   - 特徴量重要度（JSON）")
print("   - SHAP値（JSON）")


---
## ✅ Section 4 完了！

### 学んだこと
- **Snowflake ML ExperimentTracking** の使い方
- `exp.set_experiment()` で実験を作成
- `exp.start_run()` でランを開始
- `exp.log_param()`, `exp.log_metric()` でパラメータ・メトリクスを記録
- 複数実験の比較と最良モデルの選択

### 📝 記録した情報

| メソッド | 記録内容 | 用途 |
|---------|---------|------|
| `log_param()` | ハイパーパラメータ | モデル設定の追跡 |
| `log_metric()` | Accuracy, F1等 | 性能評価 |
| `log_metric()` | 特徴量重要度 | モデル解釈性 |
| `log_metric()` | SHAP値 | 予測根拠の説明 |
| `log_artifact()` | SHAP Summary Plot | 可視化の保存 |

### 作成したもの
- Experiment: `CHURN_PREDICTION_EXPERIMENT`
- 3つのRun: Baseline, DeepTree, Conservative
- 実験結果テーブル: `EXPERIMENT_RESULTS`
  - メトリクス、パラメータ、特徴量重要度、SHAP値（JSON形式）
- SHAP Summary Plot画像（アーティファクト）

### 💡 後から振り返るときのポイント

```sql
-- 実験結果テーブルから特徴量重要度を確認
SELECT 
    RUN_NAME,
    F1_SCORE,
    FEATURE_IMPORTANCE,
    SHAP_IMPORTANCE
FROM MLOPS_HOL_DB.FEATURE_STORE.EXPERIMENT_RESULTS
ORDER BY F1_SCORE DESC;
```

### 次のステップ
**Section 5: Model Registry** では、最良モデルを登録し、
バージョン管理と本番デプロイの準備を行います。

➡️ `05_MODEL_REGISTRY` Notebookに進んでください
